Load Dataset of 20 classes from cifar 100

In [ ]:
import tensorflow as tf
from keras.datasets import cifar100
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model



In [ ]:

def display_img(img_set, title_set):
    n = len(title_set)
    for i in range(n):
        plt.subplot(3, 3, i + 1)
        plt.imshow(img_set[i])
        plt.title(title_set[i])
        plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
(trainX, trainY), (testX, testY) = cifar100.load_data(label_mode='coarse')

print('trainX.shape: {}, trainY.shape: {}, testX.shape: {}, testY.shape: {}'.format(trainX.shape, trainY.shape, testX.shape, testY.shape))
print('trainX.dtype: {}, trainY.dtype: {}, testX.dtype: {}, testY.dtype: {}'.format(trainX.dtype, trainY.dtype, testX.dtype, testY.dtype))
print('trainX.Range: {} - {}, testX.Range: {} - {}'.format(trainX.max(), trainX.min(), testX.max(), testX.min()))

coarse_label_names = [
    'aquatic mammals', 'fish', 'flowers', 'food containers', 'fruit and vegetables',
    'household electrical devices', 'household furniture', 'insects', 'large carnivores',
    'large man-made outdoor things', 'large natural outdoor scenes', 'large omnivores and herbivores',
    'medium-sized mammals', 'non-insect invertebrates', 'people', 'reptiles',
    'small mammals', 'trees', 'vehicles 1', 'vehicles 2'
]

display_img(trainX[:9], [coarse_label_names[label[0]] for label in trainY[:9]])

Pre-Process the data

In [ ]:
# Normalize the pixel values to [0, 1]
trainX = trainX.astype('float32') / 255.0
testX = testX.astype('float32') / 255.0

# Resize images to 96x96 (required by most ImageNet pre-trained models)
trainX = tf.image.resize(trainX, (96, 96))
testX = tf.image.resize(testX, (96, 96))

# Convert labels to one-hot encoding (20 classes)
trainY = to_categorical(trainY, num_classes=20)
testY = to_categorical(testY, num_classes=20)

# Print data shapes and types
print(f"trainX.shape: {trainX.shape}, testX.shape: {testX.shape}")
print(f"trainX.dtype: {trainX.dtype}, testX.dtype: {testX.dtype}")
print(f"trainY.shape: {trainY.shape}, testY.shape: {testY.shape}")
print(f"trainY.dtype: {trainY.dtype}, testY.dtype: {testY.dtype}")

# Check sample labels
print("Sample trainY (one-hot encoded):")
print(trainY[:5])

7. DenseNet121

In [ ]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization

# Load DenseNet121 base model
base_model = DenseNet121(
    input_shape=(96, 96, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(20, activation='softmax')(x)

model_densenet = Model(inputs=base_model.input, outputs=predictions)
model_densenet.summary()


In [ ]:
model_densenet.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_transfer = model_densenet.fit(
    trainX, trainY,
    validation_split=0.1,
    epochs=10,
    batch_size=64,
    verbose=1
)

transfer_loss, transfer_acc = model_densenet.evaluate(testX, testY, verbose=0)
print(f"📊 DenseNet121 Transfer Learning Accuracy: {transfer_acc:.4f}")


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Unfreeze the last 40 layers of the base model
for layer in model_densenet.layers[-40:]:
    layer.trainable = True

# Recompile with a low learning rate
model_densenet.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
]

history_finetune = model_densenet.fit(
    trainX, trainY,
    validation_split=0.1,
    epochs=10,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

finetune_loss, finetune_acc = model_densenet.evaluate(testX, testY, verbose=0)
print(f"📊 DenseNet121 Fine-Tuning Accuracy: {finetune_acc:.4f}")


8. DenseNet201

In [ ]:
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization

# Load DenseNet201 base model
base_model = DenseNet201(
    input_shape=(96, 96, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze all layers initially
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(20, activation='softmax')(x)  # 20-class output

model_densenet201 = Model(inputs=base_model.input, outputs=predictions)
model_densenet201.summary()


In [ ]:
model_densenet201.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_transfer = model_densenet201.fit(
    trainX, trainY,
    validation_split=0.1,
    epochs=10,
    batch_size=64,
    verbose=1
)

transfer_loss, transfer_acc = model_densenet201.evaluate(testX, testY, verbose=0)
print(f"📊 DenseNet201 Transfer Learning Accuracy: {transfer_acc:.4f}")


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Unfreeze last 40 layers for fine-tuning
for layer in model_densenet201.layers[-40:]:
    layer.trainable = True

# Recompile with lower learning rate
model_densenet201.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
]

history_finetune = model_densenet201.fit(
    trainX, trainY,
    validation_split=0.1,
    epochs=10,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

finetune_loss, finetune_acc = model_densenet201.evaluate(testX, testY, verbose=0)
print(f"📊 DenseNet201 Fine-Tuning Accuracy: {finetune_acc:.4f}")


9. DenseNet169

In [ ]:
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization

# Load DenseNet201 base model
base_model = DenseNet169(
    input_shape=(96, 96, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze all layers initially
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(20, activation='softmax')(x)  # 20-class output

model_densenet169 = Model(inputs=base_model.input, outputs=predictions)
model_densenet169.summary()


In [ ]:
model_densenet169.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_transfer = model_densenet169.fit(
    trainX, trainY,
    validation_split=0.1,
    epochs=10,
    batch_size=64,
    verbose=1
)

transfer_loss, transfer_acc = model_densenet169.evaluate(testX, testY, verbose=0)
print(f"📊 DenseNet201 Transfer Learning Accuracy: {transfer_acc:.4f}")


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Unfreeze last 40 layers for fine-tuning
for layer in model_densenet169.layers[-40:]:
    layer.trainable = True

# Recompile with lower learning rate
model_densenet169.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
]

history_finetune = model_densenet169.fit(
    trainX, trainY,
    validation_split=0.1,
    epochs=10,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

finetune_loss, finetune_acc = model_densenet169.evaluate(testX, testY, verbose=0)
print(f"📊 DenseNet201 Fine-Tuning Accuracy: {finetune_acc:.4f}")


10. VGG19

In [ ]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization

vgg_base = VGG19(input_shape=(96, 96, 3), include_top=False, weights='imagenet')

# Freeze all layers in VGG19
for layer in vgg_base.layers:
    layer.trainable = False

# Add custom classification head
x = vgg_base.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
outputs = Dense(20, activation='softmax', name='OutputLayer')(x)

# Final model
model_vgg19 = Model(inputs=vgg_base.input, outputs=outputs, name='VGG19_CIFAR100_20Classes')
model_vgg19.summary()

In [ ]:
# Compile the model for transfer learning
model_vgg19.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train only the new head
history_vgg_transfer = model_vgg19.fit(
    trainX, trainY,
    batch_size=64,
    validation_split=0.1,
    epochs=10,
    verbose=1
)

# Evaluate
vgg_transfer_loss, vgg_transfer_acc = model_vgg19.evaluate(testX, testY, verbose=0)
print(f"📊 VGG19 Transfer Learning Test Accuracy: {vgg_transfer_acc:.4f}")

In [ ]:
# Unfreeze last few layers of VGG19 (e.g., last 4 conv blocks)
for layer in model_vgg19.layers[-40:]:
    layer.trainable = True

# Recompile with lower learning rate
model_vgg19.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune the model
history_vgg_finetune = model_vgg19.fit(
    trainX, trainY,
    batch_size=64,
    validation_split=0.1,
    epochs=10,
    verbose=1
)

# Evaluate fine-tuned model
vgg_finetune_loss, vgg_finetune_acc = model_vgg19.evaluate(testX, testY, verbose=0)
print(f"📊 VGG19 Fine-Tuning Test Accuracy: {vgg_finetune_acc:.4f}")
